# Étape 2 – Baseline frame-wise (Colab Web)

Ce notebook sert à **piloter l'entraînement de la baseline frame-wise** sur Google Colab / Colab Pro+.

La logique d'entraînement (DataLoaders, modèle, pertes, optimiseur, scheduler, mAP, etc.)
est implémentée dans `src/` dans le dépôt du projet.

Ce notebook se contente de :
1. Monter Google Drive (`MyDrive/QV-Pipe/`).
2. Cloner ou mettre à jour le dépôt dans `/content`.
3. Installer les dépendances Python.
4. Définir les chemins vers les données (frames, splits, labels).
5. Lancer le script d'entraînement `src/train/framewise_baseline.py` sur un GPU (A100 si possible).

> Avant de lancer : menu **Runtime → Change runtime type → GPU**.


In [ ]:
# Vérifier le GPU Colab disponible
!nvidia-smi

Wed Nov 26 22:02:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path

# --- Repo GitHub du projet ---
REPO_URL = "https://github.com/Simon-VDC/qv-pipe-classifier.git"  # 🔁 À adapter
PROJECT_NAME = "qv-pipe-classifier"                              # nom du dossier du projet

# --- Chemins Colab ---
COLAB_ROOT = Path("/content")
PROJECT_ROOT = COLAB_ROOT / PROJECT_NAME

# --- Chemins Drive ---
DRIVE_ROOT = Path("/content/drive/MyDrive/QV Pipe")
DATA_DIR = DRIVE_ROOT / "data"

FRAMES_DIR = DATA_DIR / "frames" / "5_forstep1and2"
SPLITS_DIR = DATA_DIR / "splits"
LABELS_PATH = DATA_DIR / "labels" / "track1-qv_pipe_train.json"

MODELS_DIR = DRIVE_ROOT / "models" / "baseline_framewise"

print("PROJECT_ROOT :", PROJECT_ROOT)
print("DRIVE_ROOT   :", DRIVE_ROOT)
print("FRAMES_DIR   :", FRAMES_DIR)
print("SPLITS_DIR   :", SPLITS_DIR)
print("LABELS_PATH  :", LABELS_PATH)
print("MODELS_DIR   :", MODELS_DIR)


PROJECT_ROOT : /content/qv-pipe-classifier
DRIVE_ROOT   : /content/drive/MyDrive/QV Pipe
FRAMES_DIR   : /content/drive/MyDrive/QV Pipe/data/frames/5_forstep1and2
SPLITS_DIR   : /content/drive/MyDrive/QV Pipe/data/splits
LABELS_PATH  : /content/drive/MyDrive/QV Pipe/data/labels/track1-qv_pipe_train.json
MODELS_DIR   : /content/drive/MyDrive/QV Pipe/models/baseline_framewise


In [ ]:
import os

if not PROJECT_ROOT.exists():
    # Clone la première fois
    !git clone {REPO_URL} {PROJECT_ROOT}
else:
    # Mise à jour si déjà présent
    %cd {PROJECT_ROOT}
    !git pull

%cd {PROJECT_ROOT}
!ls


/content/qv-pipe-classifier
Already up to date.
/content/qv-pipe-classifier
CONFIG.md      docs		exp	   project_tree.txt  requirements
data	       ENVIRONMENT.md	LICENSE    README.md	     scripts
DATA_NOTES.md  environment.yml	notebooks  reports	     src


In [ ]:
%cd {PROJECT_ROOT}

# Installer les dépendances Step 2
!pip install -r requirements/step2_training.txt


/content/qv-pipe-classifier


In [ ]:
print("Frames :", FRAMES_DIR.exists())
print("Splits :", SPLITS_DIR.exists())
print("Labels :", LABELS_PATH.exists())

print("\nContenu du dossier splits :")
!ls {SPLITS_DIR}

print("\nQuelques frames :")
!ls {FRAMES_DIR} | head -n 10


Frames : True
Splits : True
Labels : True

Contenu du dossier splits :
ls: cannot access '/content/drive/MyDrive/QV': No such file or directory
ls: cannot access 'Pipe/data/splits': No such file or directory

Quelques frames :
ls: cannot access '/content/drive/MyDrive/QV': No such file or directory
ls: cannot access 'Pipe/data/frames/5_forstep1and2': No such file or directory


In [ ]:
import sys
sys.path.append(str(PROJECT_ROOT))

try:
    import src
    print("Import de src OK ✅")
except Exception as e:
    print("Erreur d'import de src ❌")
    print(e)


Import de src OK ✅


### 🔧 Configuration de l'entraînement (variables)

Ces variables contrôlent le comportement de l'entraînement pour l'étape 2.

- **FOLD** : numéro du fold utilisé pour la validation (0 à 4).  
  Le modèle s'entraîne sur les 4 autres folds.  
  Ex. `FOLD = 0` → validation sur le fold 0, entraînement sur 1–4.

- **EPOCHS** : nombre d'itérations complètes sur les données d'entraînement.  
  Typique pour une baseline : 10 à 20 epochs.

- **BATCH_SIZE** : nombre de vidéos par batch.  
  Chaque vidéo contient 5 frames.  
  Ex. `BATCH_SIZE = 32` → 32 vidéos × 5 frames par traitement.

- **LR (Learning Rate)** : taux d’apprentissage maximal utilisé par OneCycleLR.  
  Valeur standard : 1e-3.

Ces paramètres seront ensuite passés au script d'entraînement `src/train/framewise_baseline.py`.


In [2]:
FOLD = 0
EPOCHS = 10
BATCH_SIZE = 32
LR = 1e-3

print("Configuration prête :")
print("fold =", FOLD)
print("epochs =", EPOCHS)
print("batch_size =", BATCH_SIZE)
print("learning rate =", LR)


Configuration prête :
fold = 0
epochs = 10
batch_size = 32
learning rate = 0.001
